In [55]:
import torch
import torch.nn as nn
import json
import os

In [56]:
if os.getcwd() != "/Users/user/Code/ml-bootstrap-example/":
    os.chdir("/Users/user/Code/ml-bootstrap-example/")

cwd = os.getcwd()
print(cwd)

/Users/user/Code/ml-bootstrap-example


In [57]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, data, labels):
        self.labels = torch.load("data/processed/" + labels)
        self.data = torch.load("data/processed/" + data) 

  def __len__(self):
        return len(self.data)

  def __getitem__(self, index):
        X = self.data[index]
        y = self.labels[index]
        return X, y

In [58]:
def load_evaluation_data():
    print("Loading the processed validation data...")
    validation_data = Dataset("valid_x_processed.pt", "valid_y_processed.pt")
    print("Generating data and creating batches for validation...")
    evaluation_loader = torch.utils.data.DataLoader(
                      validation_data,
                      batch_size=100, 
                      shuffle=True)
    return evaluation_loader

In [59]:
evaluation_loader = load_evaluation_data()

Loading the processed validation data...
Generating data and creating batches for validation...


In [60]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)

In [61]:
network = Net()
network.load_state_dict(torch.load('models/model.pth'))

<All keys matched successfully>

In [64]:
def eval():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in evaluation_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(evaluation_loader.dataset)
  output = {"Avg. Loss": round(test_loss, 4), "Accuracy": round((correct / len(evaluation_loader.dataset)).item(), 4)}
  output_file = open("metrics.json", "w")
  json.dump(output, output_file, indent=6)

In [65]:
eval()